In [ ]:
path = 'data/emails.txt'
with open(path, 'rb') as f:
    text = f.read().decode('utf-8', errors='replace')

In [ ]:
import re

def remove_special_characters(dirty_text):
    # The following pattern will match any character that is not a letter, number, basic punctuation, or `-` and ``
    pattern = re.compile(r'[^a-zA-Z0-9\s.,?!:;@#$%^&*()_+={}[\]|\\<>/"\'`-]', flags=re.UNICODE)
    clean_text = pattern.sub('', dirty_text)
    return clean_text

In [ ]:
def remove_minute_read(dirty_text):
    minute_read_pattern = re.compile(r'\(?\d*\s*minute\s*read\)?', re.IGNORECASE)
    clean_text = re.sub(minute_read_pattern, '', dirty_text)
    return clean_text

In [ ]:
import quopri

def decode_quoted_printable(coded_text):
    decoded_text = quopri.decodestring(coded_text.encode('utf-8', errors='replace')).decode('utf-8', errors='replace')
    return decoded_text

In [ ]:
def remove_square_brackets_urls(dirty_text):
    cleaned_text = re.sub(r'\[.*?\]', '', dirty_text)
    return cleaned_text

In [ ]:
def remove_blank_lines(dirty_text):
    lines = dirty_text.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

In [ ]:
def cleanup_text(dirty_text):
    dirty_text = decode_quoted_printable(dirty_text)
    dirty_text = remove_special_characters(dirty_text)
    dirty_text = remove_square_brackets_urls(dirty_text)
    dirty_text = remove_minute_read(dirty_text)
    clean_text = remove_blank_lines(dirty_text)
    return clean_text

text = cleanup_text(text)

In [ ]:
def extract_sections(text, start_section, end_section):
    # Define the regular expression pattern to match the desired sections
    pattern = re.compile(r'(?i){}(.*?)(?={})'.format(re.escape(start_section), re.escape(end_section)), re.MULTILINE | re.DOTALL)
    sections = re.findall(pattern, text)

    return sections

In [ ]:
arguments = [
    "Big Tech & Startups",
    "Science & Futuristic Technology",
    "Programming, Design & Data Science",
    "Miscellaneous"
]

argument_to_code = {argument: code for code, argument in enumerate(arguments)}

text_by_argument = {}

for i in range(len(arguments) - 1):
    argument = arguments[i]
    next_argument = arguments[i + 1]

    text_by_argument[argument] = extract_sections(text, argument, next_argument)

for section, content in text_by_argument.items():
    print(f"{section}: {len(content)} sections extracted")

In [ ]:
def print_first_10_sections(sections):
    for i, section in enumerate(sections[:10], start=1):
        print(section)

#print the first 10 extracted section for the BIG TECH & STARTUPS argument
print_first_10_sections(text_by_argument[arguments[0]])

In [ ]:
import pandas as pd
import re

def split_section_into_articles(section):
    pattern = r"\.(?=\s*(?:\n|\r\n?)*\s*[A-Z]{2,}(?:\s[A-Z]+)*(?!\w))"
    articles = re.split(pattern, section)
    return [article.strip() for article in articles if article.strip()]

def create_articles_dataframe(text_by_argument):
    data = []

    for argument, sections in text_by_argument.items():
        category_code = argument_to_code[argument]
        sections = text_by_argument[argument]
        for section in sections:
            articles = split_section_into_articles(section)
            for article in articles:
                article = article.replace('\n', ' ').replace('\r', ' ')
                data.append((article, category_code, argument))

    df = pd.DataFrame(data, columns=["text", "category_code", "category"])
    return df

In [ ]:
articles_df = create_articles_dataframe(text_by_argument)
articles_df

In [ ]:
def extract_title(article_text):
    title_pattern = r'^[^a-z]+'
    title_search = re.search(title_pattern, article_text)

    title = title_search.group().strip()
    last_char_of_title = title[-1]
    title = title[:-1]
    article_text = last_char_of_title + article_text[len(title_search.group()):].strip()

    return title, article_text

In [ ]:
from nltk import sent_tokenize, download

download('punkt')

def sentence_segmentation(article_text):
    title, article_text = extract_title(article_text)
    all_sentences = [title]

    sentences = sent_tokenize(article_text)
    for sentence in sentences:
        all_sentences = all_sentences + sentence.split('  ')

    return [x.strip() for x in all_sentences]

In [ ]:
def create_sentences_dataframe(articles_df):
    data = []

    for _, row in articles_df.iterrows():
        article = row['text']
        category_code = row['category_code']
        category = row['category']
        sentences = sentence_segmentation(article)

        for sentence in sentences:
            data.append((sentence, category_code, category))

    df_sentences = pd.DataFrame(data, columns=["text", "category_code", "category"])
    return df_sentences

In [ ]:
sentences_df = create_sentences_dataframe(articles_df)
sentences_df

In [ ]:
from sklearn.model_selection import train_test_split

def split_and_save_dataframes(df, name, train_ratio=0.7, validation_ratio=0.15, test_ratio=0.15):
    assert (train_ratio + validation_ratio + test_ratio == 1), "Ratios must sum up to 1"

    train_df, temp_df = train_test_split(df, stratify=df['category_code'], train_size=train_ratio)
    validation_df, test_df = train_test_split(temp_df, stratify=temp_df['category_code'], train_size=(validation_ratio / (validation_ratio + test_ratio)))

    train_df.to_pickle(f'data/{name}_training.pkl')
    validation_df.to_pickle(f'data/{name}_validation.pkl')
    test_df.to_pickle(f'data/{name}_test.pkl')

In [ ]:
articles_df.to_pickle('data/articles_df.pkl')
sentences_df.to_pickle('data/sentences_df.pkl')

In [ ]:
split_and_save_dataframes(articles_df, 'articles')
split_and_save_dataframes(sentences_df, 'sentences')